# XGBoost

Trying to learn XGBoost, test it's performance

In [1]:
import pandas as pd
import numpy as np
# from autogluon.tabular import TabularDataset, TabularPredictor

import xgboost as xgb

# from numbers import Number

In [4]:
# change number of rows/columns we will view
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

## Data Preparation

Import data

Fix NaN

Label categoricals

In [5]:
# read data and convert target variable to log
# log conversion normalizes data (also is where we measure RMSE)

# training
df_train = pd.read_csv('../data/raw/train.csv')
df_train['SalePrice']= np.log(df_train['SalePrice'])

# test
df_test = pd.read_csv('../data/raw/test.csv')
# there is no SalePrice variable (only Kaggle has)

In [6]:
# Remove NaN
for col in df_train.columns:
    if df_train[col].dtype == 'object':
        df_train[col] = df_train[col].fillna('NoFeature')
    else:
        df_train[col] = df_train[col].fillna(0)

In [7]:
# Make categorical into dummy
cat_cols = df_train.select_dtypes(include = ['object']).columns
df_encoded = pd.get_dummies(df_train, columns=cat_cols, drop_first=True)

In [8]:
X = df_encoded.drop(columns = ['SalePrice', 'Id'])
y = df_encoded['SalePrice']

In [9]:
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [26]:
# Create an XGBoost regressor model
xg_reg = xgb.XGBRegressor(
    objective ='reg:squarederror',
    colsample_bytree = 0.3,
    learning_rate = 0.1,
    max_depth = 5,
    alpha = 10,
    n_estimators = 10
)

In [27]:
# Fit the model to the training data
xg_reg.fit(X_train, y_train)

XGBRegressor(alpha=10, base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.3, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=10, n_jobs=None, num_parallel_tree=None,
             predictor=None, ...)

In [28]:
# Predict on the test set
y_pred = xg_reg.predict(X_test)

In [29]:
from sklearn.metrics import mean_squared_error

# Evaluate the model
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f'Root Mean Squared Error (RMSE): {rmse}')

Root Mean Squared Error (RMSE): 4.0201582828934335
